In [2]:
from google.cloud import bigquery
client = bigquery.Client()

DefaultCredentialsError: Could not automatically determine credentials. Please set GOOGLE_APPLICATION_CREDENTIALS or explicitly create credentials and re-run the application. For more information, please see https://cloud.google.com/docs/authentication/getting-started

In [ ]:
def load_data_from_bigquery(table, label_table):
       
    #all data
    sql = '''
    SELECT *  FROM `{}`
    '''.format(table)

    df = client.query(sql).to_dataframe()
    
    #labelled data
    sql = '''
    SELECT es.label,es.category, a.*  FROM `{}` a
    INNER JOIN `{}` es
    ON a.address = es.address
    WHERE es.label IS NOT NULL
    '''.format(table, label_table)

    dflabel = client.query(sql).to_dataframe()
    
    return df, dflabel

In [ ]:
table = "eth-tokens.test.clean_avg_with_balances_tokens"
label_table = "eth-tokens.alldata.etherscan_labelcloud"
    
df, dflabel = load_data_from_bigquery(table, label_table)

In [ ]:
SELECT contracts.address, COUNT(1) AS tx_count
FROM `bigquery-public-data.crypto_ethereum.contracts` AS contracts
JOIN `bigquery-public-data.crypto_ethereum.transactions` AS transactions ON (transactions.to_address = contracts.address)
WHERE contracts.is_erc721 = TRUE
GROUP BY contracts.address
ORDER BY tx_count DESC
LIMIT 10